# Use Case: E-commerce Search Application

In this notebook, we'll build a comprehensive e-commerce search application that brings together all the concepts we've covered so far. We'll create a modular, maintainable search system that provides powerful search capabilities for an online store.

## Introduction

Our e-commerce search application will include the following features:

1. **Basic Product Search**: Search by keywords, categories, brands, etc.
2. **Advanced Filtering**: Filter by price range, ratings, availability, etc.
3. **Smart Ranking**: Rank products based on relevance, popularity, recency, etc.
4. **Personalization**: Customize results based on user preferences and behavior
5. **Promotional Boosting**: Boost products that are on sale or featured

We'll implement these features using the Elasticsearch Query Toolkit's directive system.

## Setup

Let's import the necessary modules:

In [ ]:
import json
from elastictoolkit.queryutils.builder.directiveengine import DirectiveEngine
from elastictoolkit.queryutils.builder.functionscoreengine import FunctionScoreEngine
from elastictoolkit.queryutils.builder.matchdirective import (
    ConstMatchDirective,
    TextMatchDirective,
    RangeMatchDirective,
    FieldExistsDirective
)
from elastictoolkit.queryutils.builder.booldirective import AndDirective, OrDirective
from elastictoolkit.queryutils.builder.custommatchdirective import CustomMatchDirective
from elastictoolkit.queryutils.builder.scorefunctiondirective import (
    ScriptScoreDirective,
    FieldValueFactorDirective,
    DecayFunctionDirective,
    WeightDirective
)
from elastictoolkit.queryutils.builder.customscorefunctiondirective import CustomScoreFunctionDirective
from elastictoolkit.queryutils.builder.directivevaluemapper import DirectiveValueMapper
from elastictoolkit.queryutils.types import FieldValue, NestedField
from elastictoolkit.queryutils.consts import FieldMatchType, MatchMode, ScoreNullFilterAction, BaseMatchOp
from elasticquerydsl.filter import MatchAllQuery

# Helper function to print queries as formatted JSON
def print_query(directive):
    query = directive.to_dsl()
    print(json.dumps(query.to_query(), indent=2))

## 1. Value Mapper

First, let's define a value mapper for our e-commerce application:

In [ ]:
class EcommerceValueMapper(DirectiveValueMapper):
    # Basic product attributes
    category = FieldValue(
        fields=["category"],
        values_list=["*match_params.categories"]
    )
    brand = FieldValue(
        fields=["brand"],
        values_list=["*match_params.brands"]
    )
    search_text = FieldValue(
        fields=["name^3", "description", "keywords"],
        values_list=["match_params.search_text"]
    )
    
    # Price range
    price_range = FieldValue(
        fields=["price"],
        values_map={
            "gte": "match_params.min_price",
            "lte": "match_params.max_price"
        }
    )
    
    # Rating filter
    rating = FieldValue(
        fields=["rating"],
        values_map={"gte": "match_params.min_rating"}
    )
    
    # Availability
    in_stock = FieldValue(
        fields=["in_stock"],
        values_list=[True]
    )
    
    # Product attributes (nested)
    attributes = FieldValue(
        fields=[
            NestedField(
                field_name="attributes.name",
                nested_path="attributes"
            )
        ],
        # values_list=["*match_params.attribute_names"]
    )
    attribute_values = FieldValue(
        fields=[
            NestedField(
                field_name="attributes.value",
                nested_path="attributes"
            )
        ],
        # values_list=["*match_params.attribute_values"]
    )
    
    # Tags
    tags = FieldValue(
        fields=["tags"],
        values_list=["*match_params.tags"]
    )
    
    # Promotion fields
    on_sale = FieldValue(
        fields=["on_sale"],
        values_list=[True]
    )
    featured = FieldValue(
        fields=["featured"],
        values_list=[True]
    )

## 2. Custom Match Directives

Now, let's create some custom match directives for our e-commerce application:

In [ ]:
# Custom directive for product attributes
class ProductAttributesDirective(CustomMatchDirective):
    allowed_engine_cls_name = "EcommerceSearchEngine"
    name = "product_attributes"

    def get_directive(self):
        # Get attribute filters from match parameters
        attribute_filters = self.match_params.get("attribute_filters", {})
        if not attribute_filters:
            return None
        
        # Create a nested directive for each attribute filter
        attribute_directives = []
        for attr_name, attr_values in attribute_filters.items():
            # Create AND Directive for each attribute key-values pair
            attribute_directives.append(
                AndDirective(
                    attributes=ConstMatchDirective(
                        rule=FieldMatchType.ANY,
                        name=f"attr_{attr_name}_name"
                    ).set_values(attr_name),
                    y=ConstMatchDirective(
                        rule=FieldMatchType.ANY,
                        name=f"attr_{attr_name}_value"
                    ).set_field("attr_val").set_values(*attr_values)
                )
            )
            
        # Combine all nested directives with AND
        return AndDirective(*attribute_directives)

# Custom directive for promotional products
class PromotionalProductsDirective(CustomMatchDirective):
    allowed_engine_cls_name = "EcommerceSearchEngine"
    name = "promotional_products"
    
    def get_directive(self):
        # Check if we should include promotional products
        include_promotional = self.match_params.get("include_promotional", False)
        if not include_promotional:
            return None
        
        # Create a directive for promotional products
        return OrDirective(
            on_sale=ConstMatchDirective(rule=FieldMatchType.ANY),
            featured=ConstMatchDirective(rule=FieldMatchType.ANY)
        )

## 3. Custom Score Functions

Let's create some custom score functions for our e-commerce application:

In [ ]:
# Custom score function for popularity and recency
class PopularityRecencyScore(CustomScoreFunctionDirective):
    allowed_engine_cls_name = "EcommerceScoreEngine"
    
    def get_score_directive(self):
        # Get scoring weights from match parameters
        popularity_weight = self.match_params.get("popularity_weight", 1.0)
        recency_weight = self.match_params.get("recency_weight", 1.0)
        
        # If both weights are zero, disable the function
        if popularity_weight == 0 and recency_weight == 0:
            return None
        
        # Create a script score directive that combines popularity and recency
        return ScriptScoreDirective(
            script="""double popularity = doc['popularity_score'].value;
                    double days_old = (System.currentTimeMillis() - doc['created_date'].value) / 86400000.0;
                    double recency_factor = Math.exp(-days_old / params.recency_scale);
                    return (popularity * params.popularity_weight) + (recency_factor * params.recency_weight);""",
            weight=self._weight
        ).set_script_params(
            recency_scale=30.0,  # 30 days scale
            popularity_weight=popularity_weight,
            recency_weight=recency_weight
        )

# Custom score function for personalization
class PersonalizationScore(CustomScoreFunctionDirective):
    allowed_engine_cls_name = "EcommerceScoreEngine"
    
    def get_score_directive(self):
        # Get user preferences from match parameters
        user_preferences = self.match_params.get("user_preferences", {})
        if not user_preferences:
            return None
        
        # Extract user preferences
        preferred_categories = user_preferences.get("categories", [])
        preferred_brands = user_preferences.get("brands", [])
        viewed_products = user_preferences.get("viewed_products", [])
        
        # Create a script score directive for personalization
        return ScriptScoreDirective(
            script="""double score = 1.0;
                    
                    // Category match
                    if (params.preferred_categories.length > 0 && doc['category'].size() > 0) {
                        String category = doc['category'].value;
                        for (int i = 0; i < params.preferred_categories.length; i++) {
                            if (category == params.preferred_categories[i]) {
                                score *= params.category_boost;
                                break;
                            }
                        }
                    }
                    
                    // Brand match
                    if (params.preferred_brands.length > 0 && doc['brand'].size() > 0) {
                        String brand = doc['brand'].value;
                        for (int i = 0; i < params.preferred_brands.length; i++) {
                            if (brand == params.preferred_brands[i]) {
                                score *= params.brand_boost;
                                break;
                            }
                        }
                    }
                    
                    // Previously viewed products (slight negative boost)
                    if (params.viewed_products.length > 0) {
                        String product_id = doc['product_id'].value;
                        for (int i = 0; i < params.viewed_products.length; i++) {
                            if (product_id == params.viewed_products[i]) {
                                score *= params.viewed_product_boost;
                                break;
                            }
                        }
                    }
                    
                    return score;""",
            weight=self._weight
        ).set_script_params(
            preferred_categories=preferred_categories,
            preferred_brands=preferred_brands,
            viewed_products=viewed_products,
            category_boost=1.5,
            brand_boost=1.3,
            viewed_product_boost=0.8  # Slight negative boost for already viewed products
        )

# Custom score function for promotional boosting
class PromotionalBoostScore(CustomScoreFunctionDirective):
    allowed_engine_cls_name = "EcommerceScoreEngine"
    
    def get_score_directive(self):
        # Check if we should boost promotional products
        boost_promotional = self.match_params.get("boost_promotional", True)
        if not boost_promotional:
            return None
        
        # Create a script score directive for promotional boosting
        return ScriptScoreDirective(
            script="""double score = 1.0;
                    
                    // Boost products on sale
                    if (doc['on_sale'].value) {
                        score *= params.sale_boost;
                    }
                    
                    // Boost featured products
                    if (doc['featured'].value) {
                        score *= params.featured_boost;
                    }
                    
                    // Boost products with discount
                    if (doc['discount_percentage'].size() > 0) {
                        double discount = doc['discount_percentage'].value;
                        if (discount > 0) {
                            score *= (1.0 + (discount / 100.0) * params.discount_factor);
                        }
                    }
                    
                    return score;""",
            weight=self._weight
        ).set_script_params(
            sale_boost=1.2,
            featured_boost=1.5,
            discount_factor=2.0  # Higher discounts get higher boosts
        )

## 4. Search Engine

Now, let's create our e-commerce search engine:

In [ ]:
# Basic search engine for filtering
class EcommerceSearchEngine(DirectiveEngine):
    # Basic search and filtering
    search_text = TextMatchDirective(rule=FieldMatchType.ANY)
    category = ConstMatchDirective(rule=FieldMatchType.ANY)
    brand = ConstMatchDirective(rule=FieldMatchType.ANY, nullable_value=True)
    price_range = RangeMatchDirective()
    rating = RangeMatchDirective()
    in_stock = ConstMatchDirective(rule=FieldMatchType.ANY)
    tags = ConstMatchDirective(rule=FieldMatchType.ANY, nullable_value=True)
    
    # Custom directives
    attributes = ProductAttributesDirective()
    promotional = PromotionalProductsDirective()
    
    class Config:
        value_mapper = EcommerceValueMapper()
        match_directive_config = {
            "base_match_op": BaseMatchOp.AND  # Default is AND
        }

# Function score engine for ranking
class EcommerceScoreEngine(FunctionScoreEngine):
    # Basic scoring functions
    popularity_recency = PopularityRecencyScore(weight=2.0)
    personalization = PersonalizationScore(weight=1.5)
    promotional = PromotionalBoostScore(weight=1.2)
    
    # Boost highly rated products
    rating_boost = FieldValueFactorDirective(
        field="rating",
        factor=0.5,
        modifier="log1p",
        missing=1.0,
        weight=1.0
    )
    
    class Config:
        score_mode = "multiply"  # Multiply the scores from all functions
        boost_mode = "multiply"  # Multiply the combined function score with the query score
        value_mapper = EcommerceValueMapper()

## 5. Complete Search Application

Now, let's put everything together to create our complete e-commerce search application:

In [ ]:
def search_products(search_params):
    """Search for products using the e-commerce search application.
    
    Args:
        search_params (dict): Search parameters including filters and scoring options.
        
    Returns:
        dict: Elasticsearch query as a dictionary.
    """
    # Create the search engine for filtering
    search_engine = EcommerceSearchEngine().set_match_params(search_params)
    
    # Generate the filter query
    filter_query = search_engine.to_dsl()
    
    # Create the score engine for ranking
    score_engine = EcommerceScoreEngine().set_match_params(search_params)
    
    # Set the filter query as the base query for the score engine
    score_engine.set_match_dsl(filter_query)
    
    # Generate the final query
    final_query = score_engine.to_dsl()
    
    return final_query.to_query()

## 6. Example Searches

Let's try some example searches to see our e-commerce search application in action:

In [ ]:
# Basic search for smartphones
basic_search_params = {
    "search_text": "smartphone",
    "categories": ["electronics", "mobile phones"],
    "min_price": 300,
    "max_price": 1000,
    "min_rating": 4.0,
    "in_stock": True,
}

print("Basic Smartphone Search:")
basic_query = search_products(basic_search_params)
print(json.dumps(basic_query, indent=2))

In [ ]:
# Advanced search with attributes and promotional products
advanced_search_params = {
    "search_text": "laptop",
    "categories": ["electronics", "computers"],
    "brands": ["Apple", "Dell", "HP"],
    "min_price": 800,
    "max_price": 2000,
    "min_rating": 4.0,
    "in_stock": True,
    "attribute_filters": {
        "processor": ["Intel i7", "Intel i9", "AMD Ryzen 7"],
        "ram": ["16GB", "32GB"]
    },
    "include_promotional": True,
    "boost_promotional": True,
    "popularity_weight": 1.5,
    "recency_weight": 1.0
}

print("Advanced Laptop Search:")
advanced_query = search_products(advanced_search_params)
print(json.dumps(advanced_query, indent=2))

In [ ]:
# Personalized search with user preferences
personalized_search_params = {
    "search_text": "headphones",
    "categories": ["electronics", "audio"],
    "min_price": 50,
    "max_price": 300,
    "min_rating": 3.5,
    "in_stock": True,
    "tags": ["wireless", "bluetooth"],
    "user_preferences": {
        "categories": ["audio", "electronics"],
        "brands": ["Sony", "Bose", "Sennheiser"],
        "viewed_products": ["PROD123", "PROD456", "PROD789"]
    },
    "popularity_weight": 1.0,
    "recency_weight": 0.5
}

print("Personalized Headphones Search:")
personalized_query = search_products(personalized_search_params)
print(json.dumps(personalized_query, indent=2))

## Summary

In this notebook, we've built a comprehensive e-commerce search application that brings together all the concepts we've covered in previous notebooks. Our application includes:

1. **Value Mapping**: A comprehensive value mapper for e-commerce products
2. **Custom Match Directives**: Specialized directives for product attributes and promotional products
3. **Custom Score Functions**: Advanced scoring for popularity, recency, personalization, and promotional boosting
4. **Search Engine**: A modular search engine that combines filtering and scoring

This application demonstrates how the Elasticsearch Query Toolkit can be used to build sophisticated search applications with complex filtering and scoring requirements. The modular, declarative approach makes the code clean, maintainable, and extensible.

Key benefits of this approach include:

1. **Separation of Concerns**: Filtering logic is separated from scoring logic
2. **Reusability**: Custom directives and score functions can be reused across multiple applications
3. **Maintainability**: The declarative approach makes the code easy to understand and maintain
4. **Extensibility**: New features can be added by creating new directives or score functions
5. **Flexibility**: The application can adapt to different search requirements by changing match parameters

By using the Elasticsearch Query Toolkit, we've created a powerful, flexible search application that can handle a wide range of e-commerce search scenarios.